In [42]:
import os
import numpy as np

# Provide the path to the overall experiment folder and the path where you would like the merged file to be saved
dir = r"D:\2024-01-03_ALC4_day1"
data_path = os.path.join(dir, '02-processed')
npy_path = os.path.join(dir, '00-npy files')
raw_path = os.path.join(dir, '01-raw')

output_path = dir#raw_path

In [ ]:
#Use code below for default OpenEphys filenames and directory tree structure. 
# This block generates the list of filenames for files to be concatenated.
# Feed files_to_concat into the concatenation block below.

#Empty list will contain filepaths of recordings that are to be concatenated
files_to_concat = []

#OpenEphys files will have subdirectories for every set of experimental recordings
subdirectories = [item for item in os.listdir(dir) if os.path.isdir(os.path.join(dir, item))]
subdirectories = sorted(subdirectories, key=lambda x: int(''.join(filter(str.isdigit, x))))

for folder in subdirectories:
    node_path = os.path.join(dir, folder, 'Record Node 101').replace('\\', '/')
    if os.path.exists(node_path) and os.path.isdir(node_path):
        #nodes in subdirectories may have more than one experiment file in them.
        experiments = [item for item in os.listdir(node_path) if os.path.isdir(os.path.join(node_path, item))]
        experiments = sorted(experiments, key=lambda x: int(''.join(filter(str.isdigit, x))))

        if len(experiments) >= 1:
            #Each of these experiment files can contain multiple recordings.
            for experiment in experiments:
                exp_path = os.path.join(node_path,experiment).replace('\\', '/')
                recordings = [item for item in os.listdir(exp_path) if os.path.isdir(os.path.join(exp_path, item))]
                recordings = sorted(recordings, key=lambda x: int(''.join(filter(str.isdigit, x))))

            if len(recordings) >= 1:
                for recording in recordings:
                    recording_path = os.path.join(exp_path, recording,'continuous','Neuropix-PXI-100.ProbeA-AP','continuous.dat').replace('\\', '/')
                    files_to_concat.append(recording_path)
            else:
                print(f"No recording directories found in {exp_path}.")
        else:
            print(f"No experiment directories found in {node_path}.")        
    else:
        print(f"No 'Record Node 101' directory found in {os.path.join(dir, folder)}.")

print(len(files_to_concat))

In [ ]:
# # This block generates the list of filenames for files to be concatenated. Requires the data structure I used, so ignore it.
# #Empty list will contain filepaths of recordings that are to be concatenated
# files_to_concat = []

# #01-raw contains folders for every experiment
# experiments = [item for item in os.listdir(raw_path) if os.path.isdir(os.path.join(raw_path, item))]
# print(experiments)


# for experiment in experiments:
    

#     if experiment == '1-mechanical':
#         #files_to_concat.append(path)
#         recording_path = os.path.join(raw_path, experiment, 'Record Node 101', 'experiment1').replace('\\', '/')
#         recordings = [item for item in os.listdir(recording_path) if os.path.isdir(os.path.join(recording_path, item))]
#         recordings = sorted(recordings, key=lambda x: int(''.join(filter(str.isdigit, x))))

#         for recording in recordings:
#             path = os.path.join(recording_path, recording,'continuous','Neuropix-PXI-100.ProbeA-AP','continuous.dat').replace('\\', '/')
#             files_to_concat.append(path)

#     elif experiment == '2-acRecField' or '3-physio' or '4-5hz pulsed' or '5-20hz pulsed' or '6-40hz pulsed':
#         path = os.path.join(raw_path, experiment, 'continuous', 'Neuropix-PXI-100.ProbeA-AP', 'continuous.dat').replace('\\', '/')
#         files_to_concat.append(path)
#     else:
#         print('Invalid experiment folder detected.')

# print(files_to_concat)
# print(len(files_to_concat))

In [59]:
# input list of continuous.DAT filepaths to be concatenated. Will save a concatenated version in specified save_path.
def openephys_concat (recording_filepaths, save_path):
    raw_combined = open(os.path.join(save_path, 'combined_trial.DAT').replace('\\', '/'), 'wb')

    for recording in recording_filepaths:
        to_append = np.memmap(recording, mode='r')
        to_append.tofile(raw_combined)
            
    raw_combined.close()
    print('Concatenation complete.')

openephys_concat(files_to_concat, output_path)

Concatenation complete.


In [ ]:
#Ignore This
#Empty list will contain filepaths of recordings that are to be concatenated
mech_files = []

#01-raw contains folders for every experiment
experiments = [item for item in os.listdir(raw_path) if os.path.isdir(os.path.join(raw_path, item))]

for experiment in experiments:

    if experiment == '1-mechanical':
        #files_to_concat.append(path)
        recording_path = os.path.join(raw_path, experiment).replace('\\', '/')
        recordings = [item for item in os.listdir(recording_path) if os.path.isdir(os.path.join(recording_path, item))]
        recordings = sorted(recordings, key=lambda x: int(''.join(filter(str.isdigit, x))))

        for recording in recordings:
            path = os.path.join(recording_path, recording,'continuous','Neuropix-PXI-100.ProbeA-AP','continuous.dat').replace('\\', '/')
            mech_files.append(path)
    else:
        pass

sample_lengths = []
for file in mech_files[1:26]:

    try:
        with open(file, 'rb') as file:
            # Read the content of the binary file
            file_content = file.read()
            # Get the length of the binary data
            sample_lengths.append(len(file_content))
    except FileNotFoundError:
        print(f"File '{file}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

print((np.mean(sample_lengths)/30000/385/2))